In [1]:
from __future__ import division, print_function

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal

import matplotlib.pyplot as plt

import sys

import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')


class GAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3   #imagens passadas em grayscale com apenas um canal
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100   #dimensao do espaco latente (ruido)
        
        optimizer = Adam(0.0002, 0.5)   # taxa de aprendizado e beta1 do otimizador Adam
        
        # Criando e compilando o discriminador
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = ['binary_crossentropy'], 
                                  optimizer = optimizer,
                                  metrics = ['accuracy'])
        
        # Construindo e compilando o gerador
        self.generator = self.build_generator()
        
        # Gerador recebe ruido na entrada e gera imagens
        z = Input(shape=(self.latent_dim))
        img = self.generator(z)
        
        # Somente o gerador e treinado no modelo da GAN
        self.discriminator.trainable = False
        
        # O discriminador recebe imagens geradas e determina sua validade
        validity = self.discriminator(img)
        
        # O modelo combinado treina o gerador com base na resposta do discriminador
        self.combined = Model(z, validity)
        self.combined.compile(loss = 'binary_crossentropy',
                             optimizer = optimizer)
        
        
    def build_generator(self):
      
        init=RandomNormal(stddev=0.02)
        
        model = Sequential()
        
        model.add(Dense(256, input_dim=self.latent_dim, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization())
        model.add(Dense(512, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization())
        model.add(Dense(1024, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization())
        model.add(Dense(2048, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization())
        model.add(Dense(np.prod(self.img_shape), activation='tanh', kernel_initializer=init))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    
    def build_discriminator(self):
        
        init=RandomNormal(stddev=0.02)

        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(1024, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
                
        img = Input(shape=self.img_shape)        
        validity = model(img)
        
        return Model(img, validity)
    
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        
        (X_train, _),(_, _) = cifar10.load_data()
        
        # Reescalando dados de [0 255] (greyscale) para [-1 1]
        X_train = (X_train - 127.5)/127.5
        
        # Labels para falsos e verdadeiros
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        D_loss = []
        G_loss = []
        Acc = []
        
        for epoch in range(epochs):
            
            # Treinamento do Discriminador
            
            # Seleciona um conjunto aleatorio de imagens
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Gera um conjunto de novas imagens
            gen_imgs = self.generator.predict(noise)
            
            # Treina o discriminador
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5*np.add(d_loss_fake, d_loss_real)
            
            # Treina Gerador
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Treina Gerador (usa indices como se dados criados fossem verdadeiros para induzir erro no discriminador)
            g_loss = self.combined.train_on_batch(noise, valid)
            
            # Plot do progresso do treinamento
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            D_loss.append(d_loss[0])
            G_loss.append(g_loss)
            Acc.append(d_loss[1])
            
            # Salvando imagens geradas com certo intervalo de epocas
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.save_model()
                
        return D_loss, G_loss, Acc

            
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        # Reescalando imagens para [0 1]
        gen_imgs = 0.5*gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt+=1
        fig.savefig("/content/gdrive/My Drive/Deep Learning/gan_mlp_cifar10_2/images/%d.png" % epoch)
        plt.close()
        
        
    def save_model(self):

        def save(model, model_name):
            model_path = "/content/gdrive/My Drive/Deep Learning/gan_mlp_cifar10_2/saved_model/%s.json" % model_name
            weights_path = "/content/gdrive/My Drive/Deep Learning/gan_mlp_cifar10_2/saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")
        
if __name__ == '__main__':
    gan = GAN()
    D_loss, G_loss, Acc = gan.train(epochs=50000, batch_size=100, sample_interval=200)

Output hidden; open in https://colab.research.google.com to view.